# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 15 2022 2:45PM,250849,19,ADV80007736,"AdvaGen Pharma, Ltd",Released
1,Nov 15 2022 2:43PM,250848,10,8780026,Yusen – 3D Matrix,Released
2,Nov 15 2022 2:31PM,250847,10,8780021,Yusen – 3D Matrix,Released
3,Nov 15 2022 2:28PM,250846,16,EMERSA-8780018,"Emersa Waterbox, LLC",Released
4,Nov 15 2022 2:24PM,250845,19,ADV80007735,"AdvaGen Pharma, Ltd",Released
5,Nov 15 2022 2:24PM,250845,19,ADV80007740,"AdvaGen Pharma, Ltd",Released
6,Nov 15 2022 2:24PM,250842,10,MSP215879,"Methapharm, Inc.",Released
7,Nov 15 2022 2:24PM,250842,10,MSP215882,"Methapharm, Inc.",Released
8,Nov 15 2022 2:24PM,250842,10,MSP215885,"Methapharm, Inc.",Released
9,Nov 15 2022 2:24PM,250842,10,MSP215886,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,250845,Released,2
49,250846,Released,1
50,250847,Released,1
51,250848,Released,1
52,250849,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
250845,NaN,NaN,2.0
250846,NaN,NaN,1.0
250847,NaN,NaN,1.0
250848,NaN,NaN,1.0
250849,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0.0,0.0,48.0
250755,2.0,14.0,13.0
250757,0.0,0.0,1.0
250766,0.0,0.0,1.0
250768,0.0,50.0,9.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
250670,0,0,48
250755,2,14,13
250757,0,0,1
250766,0,0,1
250768,0,50,9


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,0,0,48
1,250755,2,14,13
2,250757,0,0,1
3,250766,0,0,1
4,250768,0,50,9


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,250670,,,48
1,250755,2,14,13
2,250757,,,1
3,250766,,,1
4,250768,,50,9


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 15 2022 2:45PM,250849,19,"AdvaGen Pharma, Ltd"
1,Nov 15 2022 2:43PM,250848,10,Yusen – 3D Matrix
2,Nov 15 2022 2:31PM,250847,10,Yusen – 3D Matrix
3,Nov 15 2022 2:28PM,250846,16,"Emersa Waterbox, LLC"
4,Nov 15 2022 2:24PM,250845,19,"AdvaGen Pharma, Ltd"
6,Nov 15 2022 2:24PM,250842,10,"Methapharm, Inc."
13,Nov 15 2022 2:23PM,250844,10,"Methapharm, Inc."
15,Nov 15 2022 2:22PM,250843,19,"AdvaGen Pharma, Ltd"
24,Nov 15 2022 2:18PM,250841,10,"Citieffe, Inc."
26,Nov 15 2022 2:13PM,250839,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Nov 15 2022 2:45PM,250849,19,"AdvaGen Pharma, Ltd",,,1
1,Nov 15 2022 2:43PM,250848,10,Yusen – 3D Matrix,,,1
2,Nov 15 2022 2:31PM,250847,10,Yusen – 3D Matrix,,,1
3,Nov 15 2022 2:28PM,250846,16,"Emersa Waterbox, LLC",,,1
4,Nov 15 2022 2:24PM,250845,19,"AdvaGen Pharma, Ltd",,,2
5,Nov 15 2022 2:24PM,250842,10,"Methapharm, Inc.",,,7
6,Nov 15 2022 2:23PM,250844,10,"Methapharm, Inc.",,,2
7,Nov 15 2022 2:22PM,250843,19,"AdvaGen Pharma, Ltd",,3,6
8,Nov 15 2022 2:18PM,250841,10,"Citieffe, Inc.",,,2
9,Nov 15 2022 2:13PM,250839,10,ISDIN Corporation,,11,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 2:45PM,250849,19,"AdvaGen Pharma, Ltd",1,,
1,Nov 15 2022 2:43PM,250848,10,Yusen – 3D Matrix,1,,
2,Nov 15 2022 2:31PM,250847,10,Yusen – 3D Matrix,1,,
3,Nov 15 2022 2:28PM,250846,16,"Emersa Waterbox, LLC",1,,
4,Nov 15 2022 2:24PM,250845,19,"AdvaGen Pharma, Ltd",2,,
5,Nov 15 2022 2:24PM,250842,10,"Methapharm, Inc.",7,,
6,Nov 15 2022 2:23PM,250844,10,"Methapharm, Inc.",2,,
7,Nov 15 2022 2:22PM,250843,19,"AdvaGen Pharma, Ltd",6,3,
8,Nov 15 2022 2:18PM,250841,10,"Citieffe, Inc.",2,,
9,Nov 15 2022 2:13PM,250839,10,ISDIN Corporation,1,11,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 2:45PM,250849,19,"AdvaGen Pharma, Ltd",1,,
1,Nov 15 2022 2:43PM,250848,10,Yusen – 3D Matrix,1,,
2,Nov 15 2022 2:31PM,250847,10,Yusen – 3D Matrix,1,,
3,Nov 15 2022 2:28PM,250846,16,"Emersa Waterbox, LLC",1,,
4,Nov 15 2022 2:24PM,250845,19,"AdvaGen Pharma, Ltd",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 2:45PM,250849,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
1,Nov 15 2022 2:43PM,250848,10,Yusen – 3D Matrix,1.0,NaN,NaN
2,Nov 15 2022 2:31PM,250847,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Nov 15 2022 2:28PM,250846,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
4,Nov 15 2022 2:24PM,250845,19,"AdvaGen Pharma, Ltd",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 15 2022 2:45PM,250849,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
1,Nov 15 2022 2:43PM,250848,10,Yusen – 3D Matrix,1.0,0.0,0.0
2,Nov 15 2022 2:31PM,250847,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Nov 15 2022 2:28PM,250846,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
4,Nov 15 2022 2:24PM,250845,19,"AdvaGen Pharma, Ltd",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6270504,166.0,78.0,5.0
12,501573,2.0,0.0,0.0
15,752253,60.0,50.0,0.0
16,501612,2.0,0.0,0.0
19,752537,9.0,3.0,0.0
20,501584,6.0,0.0,4.0
21,1253974,5.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6270504,166.0,78.0,5.0
1,12,501573,2.0,0.0,0.0
2,15,752253,60.0,50.0,0.0
3,16,501612,2.0,0.0,0.0
4,19,752537,9.0,3.0,0.0
5,20,501584,6.0,0.0,4.0
6,21,1253974,5.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,166.0,78.0,5.0
1,12,2.0,0.0,0.0
2,15,60.0,50.0,0.0
3,16,2.0,0.0,0.0
4,19,9.0,3.0,0.0
5,20,6.0,0.0,4.0
6,21,5.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,166.0
1,12,Released,2.0
2,15,Released,60.0
3,16,Released,2.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,4.0,0.0
Executing,78.0,0.0,50.0,0.0,3.0,0.0,0.0
Released,166.0,2.0,60.0,2.0,9.0,6.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,78.0,0.0,50.0,0.0,3.0,0.0,0.0
2,Released,166.0,2.0,60.0,2.0,9.0,6.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,4.0,0.0
1,Executing,78.0,0.0,50.0,0.0,3.0,0.0,0.0
2,Released,166.0,2.0,60.0,2.0,9.0,6.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()